# OverView
- This is a code of my submission.
- This is not my last submission but almost the same.
    - My last submission is little more complex, so I choose this simple one. Scores are not so different.
- My last submission is leaked and score is about 1.31, but definitely leaked.
- I think my submission score is about 1.35, same as this one.


# My Original Effort (only one thing)
- I trained model for target1 from dataset, and model for target2 from dataset+target1, model for target3 from dataset+target1+target2, model for target4 from dataset+target1+target2+target3
- I predicted targets as above by predicted ones.

# About More Efforts and My Skills
- I wanted to make things better, but I couldn't do it because I couldn't make enough time, and I don't have enough skills.
- I'm just a beginner and this is my first submission for a (ongoing) competition. So I know almost nothing about machine learning, kaggle and python. Please comment anything helpful or useful. Thank you!

# Sorry for my poor English...
- I'm sorry for my poor English. If you tell me better way of making English comments, I'm verry happy!

---

## Make Dataset

At first, I made dataset. There is nothing special, but remark that it contains some target encodings. It makes a score little better.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
import lightgbm as lgbm
import mlb
import gc

In [ ]:
BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting')
TRAIN_DIR = Path('../input/mlb-pdef-train-dataset')

In [ ]:
# loading data
players = pd.read_csv(BASE_DIR / 'players.csv')
rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
scores = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')
scores = scores.groupby(['playerId', 'date']).max().reset_index()

player_target_stats = pd.read_csv("../input/player-target-stats/player_target_stats.csv")

In [ ]:
# making data for events file, but it dosen't make a score better...
events = pd.read_pickle(TRAIN_DIR / 'events_train.pkl')

def make_events_df(df_events=events):
    
    
    game_info = ["date"]
    pitch_cols = ["pitcherId", "pitchType", "spinRate", "endSpeed", "nastyFactor"]
    bat_cols = ["hitterId", "totalDistance", "launchSpeed"]
    
    # 投手
    pitch = df_events[df_events.gameType=="R"][game_info+pitch_cols].copy()
    ## 速球について
    pitch_fb = pitch[pitch.pitchType.isin(["FF", "FT"])][["date", "pitcherId", "spinRate", "endSpeed"]]
    pitch_fb_mean = pitch_fb.groupby(["date", "pitcherId"], as_index=False).mean()
    pitch_fb_mean.rename(columns={"spinRate":"spinRate_mean", "endSpeed":"endSpeed_mean"}, inplace=True)
    pitch_fb_max = pitch_fb.groupby(["date", "pitcherId"], as_index=False).max()
    pitch_fb_max.rename(columns={"spinRate":"spinRate_max", "endSpeed":"endSpeed_max"}, inplace=True)
    pitch_fb_min = pitch_fb.groupby(["date", "pitcherId"], as_index=False).min()
    pitch_fb_min.rename(columns={"spinRate":"spinRate_min", "endSpeed":"endSpeed_min"}, inplace=True)
    ## nastyFactorについて
    pitch_nasty = pitch[["date", "pitcherId", "nastyFactor"]]
    pitch_nasty_mean = pitch_nasty.groupby(["date", "pitcherId"], as_index=False).mean()
    pitch_nasty_mean.rename(columns={"nastyFactor":"nastyFactor_mean"}, inplace=True)
    pitch_nasty_max = pitch_nasty.groupby(["date", "pitcherId"], as_index=False).max()
    pitch_nasty_max.rename(columns={"nastyFactor":"nastyFactor_max"}, inplace=True)
    pitch_nasty_min = pitch_nasty.groupby(["date", "pitcherId"], as_index=False).min()
    pitch_nasty_min.rename(columns={"nastyFactor":"nastyFactor_min"}, inplace=True)
    ## 結合
    events_pitch = pitch_fb_mean.merge(pitch_fb_max, on=["date", "pitcherId"])
    events_pitch = events_pitch.merge(pitch_fb_min, on=["date", "pitcherId"])
    events_pitch = events_pitch.merge(pitch_nasty_mean, on=["date", "pitcherId"])
    events_pitch = events_pitch.merge(pitch_nasty_max, on=["date", "pitcherId"])
    events_pitch = events_pitch.merge(pitch_nasty_min, on=["date", "pitcherId"])
    ## 名前変更
    events_pitch.rename(columns={"pitcherId":"playerId"}, inplace=True)

    
    # 打者
    bat = df_events[(df_events.gameType=="R")&(df_events.launchAngle>5)][game_info+bat_cols]
    events_bat = bat.groupby(["date", "hitterId"], as_index=False).max()
    events_bat.rename(columns={"hitterId":"playerId"}, inplace=True)
    
    del df_events, pitch, pitch_fb, pitch_fb_mean, pitch_fb_max, pitch_fb_min, 
    del pitch_nasty, pitch_nasty_mean, pitch_nasty_max, pitch_nasty_min
    del bat
    gc.collect()
    
    
    return events_pitch.merge(events_bat, on=["playerId", "date"], how="outer")

events_train = make_events_df(events)

In [ ]:
# make chosen columns
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date']

feature_cols = [
    'label_playerId', 'label_primaryPositionName', 'label_teamId',
    'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
    'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
    'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
    'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
    'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
    'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
    'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
    'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
    'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
    'groundOutsPitching', 'runsPitching', 'doublesPitching',
    'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
    'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
    'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
    'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
    'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
    'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
    'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
    'inheritedRunnersScored', 'catchersInterferencePitching',
    'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
    'assists', 'putOuts', 'errors', 'chances','target1_mean',
    'target1_median',
    'target1_std',
    'target1_min',
    'target1_max',
    'target1_prob',
    'target2_mean',
    'target2_median',
    'target2_std',
    'target2_min',
    'target2_max',
    'target2_prob',
    'target3_mean',
    'target3_median',
    'target3_std',
    'target3_min',
    'target3_max',
    'target3_prob',
    'target4_mean',
    'target4_median',
    'target4_std',
    'target4_min',
    'target4_max',
    'target4_prob',
    
    'spinRate_mean',
    'endSpeed_mean',
    'spinRate_max',
    'endSpeed_max',
    'spinRate_min',
    'endSpeed_min',
    'nastyFactor_mean',
    'nastyFactor_max',
    'nastyFactor_min',
    'totalDistance',
    'launchSpeed'
]

In [ ]:
# creat dataset
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])

del targets, rosters, scores
gc.collect()


train = train.merge(events_train, on=["date", "playerId"], how="left")

del events_train
gc.collect()



# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)

In [ ]:
# split train and valid
train_X = train[feature_cols]
train_y = train[['target1', 'target2', 'target3', 'target4']]


_index = (train['date'] < 20210401)
x_train = train_X.loc[_index].reset_index(drop=True)
y_train = train_y.loc[_index].reset_index(drop=True)
x_valid = train_X.loc[~_index].reset_index(drop=True)
y_valid = train_y.loc[~_index].reset_index(drop=True)

del train, train_X, train_y
gc.collect()

# Make Models

- As my first comment above, my effort is
    - I trained model for target1 from dataset, and model for target2 from dataset+target1, model for target3 from dataset+target1+target2, model for target4 from dataset+target1+target2+target3
    - I predict targets as above by predicted ones.
- It may be bad way for validation, but I don't know much about it. **Please tell me how did you make it or/and tell me good information or books for making good validation.**
- Parameters are not optimized by these models. These parameters are from other persons ones.

In [ ]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, x_test, params: dict=None, verbose=100, t_num=None):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = lgbm.LGBMRegressor(**params)
    model.fit(x_train, y_train, 
        eval_set=[(x_valid, y_valid)],  
        early_stopping_rounds=verbose, 
        verbose=verbose)
    oof_pred = model.predict(x_test)
    score = mean_absolute_error(oof_pred, y_valid)
    print('mae:', score)
    return oof_pred, model, score


# training lightgbm

params1 = {
    'objective':'mae',
    'reg_alpha': 0.14947461820098767, 
    'reg_lambda': 0.10185644384043743, 
    'n_estimators': 3633, 
    'learning_rate': 0.08046301304430488, 
    'num_leaves': 674, 
    'feature_fraction': 0.9101240539122566, 
    'bagging_fraction': 0.9884451442950513, 
    'bagging_freq': 8, 
    'min_child_samples': 51,
    'random_state': 42,
}

params2 = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 1000,#80,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 22,
}

params3 = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 10000,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 100,
}

params4 = {
    'objective':'mae',
    'reg_alpha': 0.016468100279441976, 
    'reg_lambda': 0.09128335764019105, 
    'n_estimators': 9868, 
    'learning_rate': 0.10528150510326864, 
    'num_leaves': 157, 
    'feature_fraction': 0.5419185713426886, 
    'bagging_fraction': 0.2637405128936662, 
    'bagging_freq': 19, 
    'min_child_samples': 71,
    'random_state': 42,
}


oof1_eval, model1_eval, score1_eval = fit_lgbm(
    x_train, y_train['target1'],
    x_valid, y_valid['target1'],
    x_valid,
    params1
)

x_train2 = x_train.copy()
x_train2["target1"] = y_train["target1"]
x_valid2, x_test2 = x_valid.copy(), x_valid.copy()
x_valid2["target1"] = y_valid["target1"]
x_test2["target1"] = oof1_eval

oof2_eval, model2_eval, score2_eval = fit_lgbm(
    x_train2, y_train['target2'],
    x_valid2, y_valid['target2'],
    x_test2,
    params2
)


x_train3 = x_train2.copy()
x_train3["target2"] = y_train["target2"]
x_valid3, x_test3 = x_valid2.copy(), x_valid2.copy()
x_valid3["target2"] = y_valid["target2"]
x_test3["target2"] = oof2_eval

del x_train2, x_valid2, x_test2
gc.collect()

oof3_eval, model3_eval, score3_eval = fit_lgbm(
    x_train3, y_train['target3'],
    x_valid3, y_valid['target3'],
    x_test3,
    params3
)

x_train4 = x_train3.copy()
x_train4["target3"] = y_train["target3"]
x_valid4, x_test4 = x_valid3.copy(), x_valid3.copy()
x_valid4["target3"] = y_valid["target3"]
x_test4["target3"] = oof3_eval

del x_train3, x_valid3, x_test3
gc.collect()

oof4_eval, model4_eval, score4_eval = fit_lgbm(
    x_train4, y_train['target4'],
    x_valid4, y_valid['target4'],
    x_test4,
    params4
)

params_dict = {
    "target1":params1, "target2":params2, "target3":params3, "target4":params4, 
}
models_eval = {
    "target1":model1_eval, "target2":model2_eval, "target3":model3_eval, "target4":model4_eval, 
}


del x_train4, x_valid4, x_test4
gc.collect()


score_eval = (score1_eval+score2_eval+score3_eval+score4_eval) / 4
print(f'score: {score_eval}')

In [ ]:
# models trained by all data
for i in range(1, 4+1):
    target_name = "target" + str(i)
    params_dict[target_name]["n_estimators"] = models_eval[target_name].best_iteration_

    
del oof1_eval, model1_eval, score1_eval, oof2_eval, model2_eval, score2_eval
del oof3_eval, model3_eval, score3_eval, oof4_eval, model4_eval, score4_eval
gc.collect()

    
x_train_all = x_train.append(x_valid)
y_train_all = y_train.append(y_valid)

del x_train, y_train, x_valid, y_valid
gc.collect()


oof1, model1, score1 = fit_lgbm(
    x_train_all, y_train_all['target1'],
    x_train_all, y_train_all['target1'],
    x_train_all,
    params_dict['target1'],
    verbose=100
)

x_train_all2,  x_test_all2 = x_train_all.copy(), x_train_all.copy()
x_train_all2["target1"] = y_train_all["target1"]
x_test_all2["target1"] = oof1


oof2, model2, score2 = fit_lgbm(
    x_train_all2, y_train_all['target2'],
    x_train_all2, y_train_all['target2'],
    x_test_all2,
    params_dict['target2'],
    verbose=100
)


x_train_all3,  x_test_all3 = x_train_all2.copy(), x_train_all2.copy()
x_train_all3["target2"] = y_train_all["target2"]
x_test_all3["target2"] = oof2

del x_train_all2, x_test_all2
gc.collect()

oof3, model3, score3 = fit_lgbm(
    x_train_all3, y_train_all['target3'],
    x_train_all3, y_train_all['target3'],
    x_test_all3,
    params_dict['target3'],
    verbose=100
)


x_train_all4,  x_test_all4 = x_train_all3.copy(), x_train_all3.copy()
x_train_all4["target3"] = y_train_all["target3"]
x_test_all4["target3"] = oof3

del x_train_all3, x_test_all3
gc.collect()

oof4, model4, score4 = fit_lgbm(
    x_train_all4, y_train_all['target4'],
    x_train_all4, y_train_all['target4'],
    x_test_all4,
    params_dict['target4'],
    verbose=100
)

del x_train_all4, x_test_all4
gc.collect()

score = (score1 + score2 + score3 + score4) / 4
print(f'score: {score}')

## Make Predictions

In [ ]:
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ["date", 'playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']
events_cols =[
    'date', 'playerId', 'spinRate_mean', 'endSpeed_mean', 'spinRate_max',
    'endSpeed_max', 'spinRate_min', 'endSpeed_min', 'nastyFactor_mean',
    'nastyFactor_max', 'nastyFactor_min', 'totalDistance', 'launchSpeed'
]


null = np.nan
true = True
false = False

env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    
    # creat dataset
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters_cols:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
    if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
        test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
        test_scores["date"] = test_df.index[0]
    else:
        test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in scores_cols:
            if col == 'playerId':
                pass
            elif col == "date":
                test_scores["date"] = test_df.index[0]
            else:
                test_scores[col] = np.nan
    
    
    if test_df['events'].iloc[0] == test_df['events'].iloc[0]:
        test_events = pd.DataFrame(eval(test_df['events'].iloc[0]))
        test_events["date"] = test_df.index[0]
        test_events = make_events_df(test_events)
    else:
        test_events = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in events_cols:
            if col == 'playerId':
                pass
            elif col == "date":
                test_events["date"] = test_df.index[0]
            else:
                test_events[col] = np.nan
    
    
    
    test_scores = test_scores.groupby(['playerId', "date"]).max().reset_index()
    test = sample_prediction_df[['playerId']].copy()
    test = test.merge(players[players_cols], on='playerId', how='left')
    test = test.merge(test_rosters[rosters_cols], on='playerId', how='left')
    test = test.merge(test_scores[scores_cols], on='playerId', how='left')
    test = test.merge(player_target_stats, on=["playerId"], how='left')
    
    test = test.merge(test_events, on=["date", "playerId"], how='left')
    del test_events
    gc.collect()

    
    test['label_playerId'] = test['playerId'].map(player2num)
    test['label_primaryPositionName'] = test['primaryPositionName'].map(position2num)
    test['label_teamId'] = test['teamId'].map(teamid2num)
    test['label_status'] = test['status'].map(status2num)
    
    test_X = test[feature_cols]
    
    
    
    # predict
    pred1 = model1.predict(test_X)
    
    test_X2 = test_X.copy()
    test_X2["target1"] = pred1
    pred2 = model2.predict(test_X2)
    
    test_X3 = test_X2.copy()
    test_X3["target2"] = pred2
    del test_X2
    gc.collect()
    pred3 = model3.predict(test_X3)
    
    test_X4 = test_X3.copy()
    test_X4["target3"] = pred3
    del test_X3
    gc.collect()
    pred4 = model4.predict(test_X4)
    
    del test_X4
    gc.collect()
    
    # merge submission
    sample_prediction_df['target1'] = np.clip(pred1, 0, 100)
    sample_prediction_df['target2'] = np.clip(pred2, 0, 100)
    sample_prediction_df['target3'] = np.clip(pred3, 0, 100)
    sample_prediction_df['target4'] = np.clip(pred4, 0, 100)
    sample_prediction_df = sample_prediction_df.fillna(0.)
    del sample_prediction_df['playerId']
    
    env.predict(sample_prediction_df)